<a href="https://colab.research.google.com/github/chapSKor/basicRAGs/blob/main/RAG_FAISS_t5Large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch transformers sentence-transformers faiss-cpu datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
import faiss
import numpy as np
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from transformers import T5ForConditionalGeneration, T5Tokenizer

"arxiv": Contains abstracts of research papers from arXiv, including agriculture, climate change, and environmental science.
"cc_news": Large dataset of news articles, useful for retrieving current events related to agriculture and the environment.
"scientific_papers": Contains scientific abstracts and papers, useful for environmental studies and agricultural research.
"ag_news": Focused on agriculture news.
"climate_fever": A dataset with climate change-related claims and evidence.
"dbpedia_14": Contains structured data from Wikipedia, which includes entries on agriculture, climate, and the environment.
"wiki_bio": Contains biographical information and relevant entries on scientists, environmentalists, etc.

In [3]:
# Step 1: Define agricultural and environmental datasets to load
datasets_to_load = [
    ('arxiv', 'train'),               # Scientific abstracts including agriculture
    ('cc_news', 'train'),             # News articles including environment
    ('scientific_papers', 'train'),   # Scientific abstracts and papers
    ('ag_news', 'train'),             # Agriculture news
    ('climate_fever', None),          # Climate change evidence
    ('dbpedia_14', None),             # Structured data including agriculture and environment
    ('wiki_bio', None)                # Biographical data on scientists and environmentalists
]

corpus = []

In [4]:
# Load multiple datasets and handle potential errors
print("Loading datasets...")
for dataset_name, config in datasets_to_load:
    try:
        if config:
            dataset = load_dataset(dataset_name, config, split='train')
        else:
            dataset = load_dataset(dataset_name, split='train')

        # Extract text fields and prioritize non-question fields
        if 'abstract' in dataset.column_names:
            corpus.extend(dataset['abstract'])
        elif 'text' in dataset.column_names:
            corpus.extend(dataset['text'])
        elif 'content' in dataset.column_names:
            corpus.extend(dataset['content'])
        elif 'claim' in dataset.column_names:
            corpus.extend(dataset['claim'])
        print(f"Loaded dataset: {dataset_name}")

    except Exception as e:
        print(f"Failed to load dataset {dataset_name}: {e}")

if not corpus:
    raise ValueError("No data loaded. Please check the dataset names and configurations.")


Loading datasets...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Failed to load dataset arxiv: Dataset 'arxiv' doesn't exist on the Hub or cannot be accessed.


README.md:   0%|          | 0.00/7.57k [00:00<?, ?B/s]

Failed to load dataset cc_news: BuilderConfig 'train' not found. Available: ['plain_text']


...

wikipedia-biography-dataset.zip:   0%|          | 0.00/334M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/582659 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/72831 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/72831 [00:00<?, ? examples/s]

Loaded dataset: wiki_bio


In [ ]:
# Step 2: Initialize the Sentence Transformer and encode the corpus
print("Encoding the corpus with Sentence Transformer...")
retriever_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
corpus_embeddings = retriever_model.encode(corpus, convert_to_tensor=False)
corpus_embeddings = np.array(corpus_embeddings).astype('float32')

Encoding the corpus with Sentence Transformer...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Step 3: Build the FAISS index
print("Building the FAISS index...")
index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

In [ ]:
# Step 4: Load T5-large model and tokenizer
print("Loading T5-large model...")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-large")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-large")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
t5_model.to(device)

In [ ]:
# Function to retrieve relevant passages using FAISS
def retrieve_passages_faiss(query, top_k=5):
    query_embedding = retriever_model.encode(query, convert_to_tensor=False).astype('float32')
    _, top_k_indices = index.search(np.array([query_embedding]), top_k)
    retrieved_passages = [corpus[idx] for idx in top_k_indices[0]]
    return retrieved_passages

# Function to generate a response using T5-large
def generate_response_with_t5(query, retrieved_passages):
    context = " ".join(retrieved_passages)
    input_text = f"question: {query} context: {context}"

    # Encode the input text for T5
    input_ids = t5_tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True).to(device)

    # Generate response using T5
    output_ids = t5_model.generate(
        input_ids,
        max_length=400,
        min_length=150,
        num_beams=7,
        early_stopping=True,
        temperature=0.6,
        top_p=0.9
    )

    response = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

In [ ]:
# Example Usage
query = "What are the best practices for sustainable agriculture?"
retrieved_passages = retrieve_passages_faiss(query, top_k=5)
response = generate_response_with_t5(query, retrieved_passages)

# Output results
print("\nQuery:", query)
print("\nRetrieved Passages:")
for passage in retrieved_passages:
    print("-", passage)
print("\nGenerated Response:", response)